In [1]:
import ray
import torch
from rllib_setup import get_env_continuous

In [2]:
ray.init()

2023-07-13 16:39:00,035	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.13
Ray version:,2.5.1
Dashboard:,http://127.0.0.1:8265


In [3]:
env = get_env_continuous()

In [4]:
env_name = "VJS"

In [5]:
ray.rllib.utils.check_env(env)

ValueError: Traceback (most recent call last):
  File "/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/utils/pre_checks/env.py", line 294, in check_multiagent_environments
    obs_and_infos = env.reset(seed=42, options={})
  File "/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/env/wrappers/pettingzoo_env.py", line 139, in reset
    info = self.env.reset(seed=seed, return_info=True, options=options)
TypeError: reset() got an unexpected keyword argument 'return_info'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/utils/pre_checks/env.py", line 77, in check_env
    check_multiagent_environments(env)
  File "/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/utils/pre_checks/env.py", line 299, in check_multiagent_environments
    raise ValueError(
ValueError: Your environment (<PettingZooEnv instance>) does not abide to the new gym